# Stock prediction: 1D convolution

In [1]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import InputLayer

from tensorflow.keras.utils import plot_model

from utils import *

# for Robust location estimation
from statsmodels import robust
import statsmodels.api as sm
from astropy.stats import biweight_location

tf.random.set_seed(25)

In [21]:
prices = pd.read_csv("data/SX5E/close_mat.csv")
composition = pd.read_csv("data/SX5E/SX5E Index_compositionMat.csv")
capitalization = pd.read_csv("data/SX5E/capitalization_mat.csv")

prices = setup_tables(prices)
composition = setup_tables(composition)
capitalization = setup_tables(capitalization)

prices = select_time_slice(prices, start=20140101, end=20181231)
capitalization = select_time_slice(capitalization, start=20140101, end=20181231)
composition = select_time_slice(composition, start=20140101, end=20181231)

prices.set_index("Date", drop=True, inplace=True)
composition.set_index("Date", drop=True, inplace=True)
capitalization.set_index("Date", drop=True, inplace=True)

In [22]:
stocks = get_full_time_stock_in_period(composition.iloc[:, 1:])
print("Number of stocks =", len(stocks))

Number of stocks = 40


In [23]:
trading_days = get_trading_dates(start_period="2014-01-01", end_period="2018-12-31", market="EUREX")
print("Total number of trading days =", len(trading_days))

Total number of trading days = 1272


In [27]:
prices = prices[stocks].loc[trading_days]
capitalization = capitalization[stocks].loc[trading_days]
composition = composition[stocks].loc[trading_days]

In [28]:
prices

,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,ENEL IM Equity,CS FP Equity,...,GLE FP Equity,ITX SQ Equity,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,ABI BB Equity,DTE GY Equity
2014-01-02,172.511642,16.715000,98.458710,55.540001,25.125425,1.7830,74.811279,118.286003,3.134,19.715000,...,41.520000,23.719999,125.599998,8.7530,67.760002,61.669998,10.080,17.320000,75.610001,12.130
2014-01-03,172.511642,16.745001,99.196777,55.680000,25.242702,1.8090,75.845627,118.060699,3.178,19.790001,...,41.325001,23.780001,126.550003,8.7250,67.750000,61.709999,10.085,17.480000,76.459999,12.270
2014-01-06,171.005936,16.545000,98.753929,55.770000,25.439726,1.8280,75.945084,115.852692,3.186,19.750000,...,41.709999,23.660000,125.400002,8.8180,66.050003,62.000000,9.981,17.580000,75.089996,12.190
2014-01-07,172.081436,16.750000,99.787231,57.369999,25.641443,1.8880,75.855568,115.041588,3.226,20.250000,...,43.389999,23.889999,126.099998,9.3210,65.519997,61.509998,10.335,17.660000,76.599998,12.250
2014-01-08,172.210495,16.850000,98.507912,58.189999,25.688353,1.9290,74.920685,113.374321,3.242,20.389999,...,44.680000,23.730000,123.750000,9.3860,65.699997,62.549999,10.555,17.370001,76.970001,12.225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,123.296356,12.510000,61.389999,39.500000,28.887671,1.9122,75.268776,253.449997,5.186,18.796000,...,28.090000,22.750000,200.699997,4.6185,132.979996,86.419998,9.490,13.774000,59.580002,15.010
2018-12-24,123.296356,12.280000,61.389999,38.790001,28.498310,1.9122,73.836601,245.149994,5.186,18.768000,...,27.610001,22.250000,197.649994,4.5580,130.880005,86.419998,9.319,13.774000,58.459999,15.010
2018-12-27,118.254364,12.020000,59.160000,38.544998,28.432636,1.9090,73.359215,247.149994,5.048,18.540001,...,27.215000,22.190001,195.600006,4.5450,132.619995,85.790001,9.191,13.520000,57.349998,14.590
2018-12-28,119.527763,12.275000,60.560001,39.375000,28.798540,1.9398,74.811279,252.750000,5.044,18.780001,...,27.795000,22.389999,198.100006,4.6410,136.619995,86.930000,9.358,13.748000,57.500000,14.820


In [31]:
p = prices.iloc[:3, :2]
c = composition.iloc[:3, :2]
cap = capitalization.iloc[:3, :2]

In [43]:
p

,VOW3 GY Equity,ENGI FP Equity
2014-01-02,172.511642,16.715000
2014-01-03,172.511642,16.745001
2014-01-06,171.005936,16.545000


In [44]:
c

,VOW3 GY Equity,ENGI FP Equity
2014-01-02,1.0,1.0
2014-01-03,1.0,1.0
2014-01-06,1.0,1.0


In [36]:
w = c * cap
display(w)

,VOW3 GY Equity,ENGI FP Equity
2014-01-02,91287.281250,40330.351562
2014-01-03,91006.945312,40402.738281
2014-01-06,90532.140625,39920.171875


In [42]:
w.sum(axis=1)

2014-01-02    131617.632812
2014-01-03    131409.683594
2014-01-06    130452.312500
dtype: float64